In [ ]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers datasets

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
# argment setting
task = "ynat"
model_checkpoint = "klue/roberta-base"
batch_size = 256

In [2]:
from datasets import load_dataset
dataset = load_dataset('klue', 'ynat') # klue 의 task=nil 을 load
dataset # dataset 구조 확인

Found cached dataset klue (C:/Users/clee/.cache/huggingface/datasets/klue/ynat/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date'],
        num_rows: 45678
    })
    validation: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date'],
        num_rows: 9107
    })
})

In [10]:
label_names = dataset["train"].features["label"].names
label_names

['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']

In [3]:
dataset['train'][0]

{'guid': 'ynat-v1_train_00000',
 'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영',
 'label': 3,
 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0008508947',
 'date': '2016.06.30. 오전 10:36'}

In [4]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(dataset["train"])

,guid,title,label,url,date
0,ynat-v1_train_42776,로열브루나이 인천브루나이 왕복권 49만원대 판매,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=237&oid=001&aid=0009851400,2018.02.01. 오전 9:44
1,ynat-v1_train_15970,수도권·제주·남부에 오전 11시 폭염주의보 발효,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=248&oid=001&aid=0009430743,2017.07.25. 오전 7:48
2,ynat-v1_train_39754,신간 방법에 반대한다,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=243&oid=001&aid=0011288870,2019.12.20. 오전 10:41
3,ynat-v1_train_15645,두꺼운 외투 입으세요…수도권 올가을 첫 한파주의보,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=248&oid=001&aid=0008806870,2016.11.08. 오전 5:42
4,ynat-v1_train_36106,춘천 막국수닭갈비축제 늦더위 피해 연다,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=237&oid=001&aid=0009093901,2017.03.09. 오전 11:06
5,ynat-v1_train_02654,프랑스 노동법 개정 반대 시위 8일째…원전 노조 파업 동참종합,세계,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=104&sid2=233&oid=001&aid=0008430506,2016.05.26. 오후 6:55
6,ynat-v1_train_04361,한국당 국회 정개특위 간사에 장제원 의원 선임,정치,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=001&aid=0010537206,2018.12.20. 오전 11:24
7,ynat-v1_train_18208,대신증권 삼성그룹 삼성물산 시총 극대화할 것,경제,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=258&oid=001&aid=0008213888,2016.02.26. 오전 8:56
8,ynat-v1_train_00251,LG와 만나는 김태형 두산 감독 지금은 이기는 게 목표 아냐,스포츠,https://sports.news.naver.com/news.nhn?oid=001&aid=0010369375,2018.09.29 16:37
9,ynat-v1_train_04785,강릉 승용차 추락 사고…신원파악 한때 혼선,사회,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=257&oid=001&aid=0010718933,2019.03.26. 오후 2:08


In [5]:
import torch
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

C:\Users\clee\venv10\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\clee\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [6]:
def preprocess_function(examples):
    return tokenizer(examples['title'], truncation=True)

In [7]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/45678 [00:00<?, ? examples/s]

Map:   0%|          | 0/9107 [00:00<?, ? examples/s]

In [13]:
num_labels = dataset['train'].features['label'].num_classes

id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

{0: 'IT과학', 1: '경제', 2: '사회', 3: '생활문화', 4: '세계', 5: '스포츠', 6: '정치'}


{'IT과학': 0, '경제': 1, '사회': 2, '생활문화': 3, '세계': 4, '스포츠': 5, '정치': 6}

In [10]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
num_labels = 7 # label 개수는 task 마다 달라질 수 있음

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initial

In [11]:
# import os
#
# model_name = model_checkpoint.split("/")[-1]
# output_dir = os.path.join("test-klue", "ynat") # task 별로 바꿔줄 것
# logging_dir = os.path.join(output_dir, 'logs')
output_dir = "klue_ynat_roberta_base_model"
args = TrainingArguments(
    # checkpoint, 모델의 checkpoint 가 저장되는 위치
    output_dir=output_dir,
    overwrite_output_dir=True,

    # Model Save & Load
    save_strategy = "epoch", # 'steps'
    load_best_model_at_end=True,
    save_steps = 500,


    # Dataset, epoch 와 batch_size 선언
    num_train_epochs=5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    # Optimizer
    learning_rate=2e-5, # 5e-5
    weight_decay=0.01,  # 0
    # warmup_steps=200,

    # Resularization
    # max_grad_norm = 1.0,
    # label_smoothing_factor=0.1,


    # Evaluation
    evaluation_strategy = "epoch",

    # HuggingFace Hub Upload, 모델 포팅을 위한 인자
    push_to_hub=True,

    # Randomness, 재현성을 위한 rs 설정
    seed=42,
)

또한 task 별로 metric 지정이 필요합니다. YNAT task 는 Macro-F1을 평가지표로 사용합니다. 다음으로 trainer 객체를 정의하겠습니다. 우선 metric 설정이 필요합니다. datasets 라이브러리에서 제공하는 evaluation metric의 리스트를 확인하겠습니다.

In [12]:
# metric list 확인
from datasets import list_metrics, load_metric
metrics_list = list_metrics()
len(metrics_list)
print(', '.join(metric for metric in metrics_list))

C:\Users\clee\AppData\Local\Temp\ipykernel_9948\2262110893.py:3: FutureWarning: list_metrics is deprecated and will be removed in the next major version of datasets. Use 'evaluate.list_evaluation_modules' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metrics_list = list_metrics()


accuracy, bertscore, bleu, bleurt, brier_score, cer, character, charcut_mt, chrf, code_eval, comet, competition_math, coval, cuad, exact_match, f1, frugalscore, glue, google_bleu, indic_glue, mae, mahalanobis, mape, mase, matthews_correlation, mauve, mean_iou, meteor, mse, nist_mt, pearsonr, perplexity, poseval, precision, r_squared, recall, rl_reliability, roc_auc, rouge, sacrebleu, sari, seqeval, smape, spearmanr, squad, squad_v2, super_glue, ter, trec_eval, wer, wiki_split, xnli, xtreme_s, AlhitawiMohammed22/CER_Hu-Evaluation-Metrics, BucketHeadP65/confusion_matrix, BucketHeadP65/roc_curve, Drunper/metrica_tesi, Felipehonorato/eer, He-Xingwei/sari_metric, JP-SystemsX/nDCG, Josh98/nl2bash_m, Kyle1668/squad, Muennighoff/code_eval, NCSOFT/harim_plus, Natooz/ece, NikitaMartynov/spell-check-metric, Pipatpong/perplexity, Splend1dchan/cosine_similarity, Viona/fuzzy_reordering, Viona/kendall_tau, Vipitis/shadermatch, Yeshwant123/mcc, abdusah/aradiawer, abidlabs/mean_iou, abidlabs/mean_iou2,

In [13]:
# YNAT의 metric은 F1 score를 사용합니다.
metric_macrof1 = load_metric('f1')

def compute_metrics(eval_pred):
    predictions = eval_pred.predictions.argmax(-1)
    labels = eval_pred.label_ids
    return metric_macrof1.compute(predictions=predictions,
                                  references=labels, average='macro')

C:\Users\clee\AppData\Local\Temp\ipykernel_9948\2202666879.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric_macrof1 = load_metric('f1')


In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()